In [103]:
def get_input() -> list[str]:
    return get_lines_from_file('./input')

def get_test_input() -> list[str]:
    return get_lines_from_file('./test_input')

def get_lines_from_file(filepath) -> list[str]:
    with open(filepath) as f:
        return [line.strip('\n') for line in f.readlines()]

def get_str_from_file(filepath) -> str:
    with open(filepath) as f:
        return f.readline().strip('\n')

def get_int_from_file(filepath) -> int:
    with open(filepath) as f:
        return int(f.readline().strip())

def log_invocation(func):
    def logged_func(*args):
        res = func(*args)
        print(f'{func.__name__}({args}) -> {res}')
        return res
    return logged_func

In [104]:
from typing import Tuple
import re

class Node:
    def __init__(self, name, parent, children) -> None:
        self.name = name
        self.parent: Node = parent
        self.children: list[Node] = children            

class File(Node):
    def __init__(self, name, parent: 'Directory', size: int) -> None:
        super().__init__(name, parent, [])
        self.size = size
    
    def __str__(self) -> str:
        return f'{self.name} (file, size={self.size})'

    def __repr__(self) -> str:
        return f'File({self.name})'

class Directory(Node):
    def __init__(self, name, parent) -> None:
        super().__init__(name, parent, children=[])

    @property
    def size(self) -> int:
        size = 0
        for child in self.children:
            size += child.size
        return size

    def cd(self, target) -> 'Directory':
        children_names = list(map(lambda c: c.name, self.children))
        if target == self.name and target not in children_names:
            return self
        elif target == '..':
            return self.parent #if self.parent is not None else self
        else:
            matching_child = next(filter(lambda e: e.name == target, self.children), None)
            if matching_child is not None:
                return matching_child
            else:
                new_dir = Directory(target, self)
                self.children.append(new_dir)
                return new_dir

    def add_child(self, file: 'File'):
        self.children.append(file)

    def collect_subdirectories(self) -> list['Directory']:
        result = []
        sub_dirs = list(filter(lambda e: isinstance(e, Directory), self.children))
        for dir in sub_dirs:
            result += dir.collect_subdirectories()
        return result + sub_dirs

    def __str__(self) -> str:
        return f'{self.name} (dir, size={self.size})'

    def __repr__(self) -> str:
        return f'Directory({self.name})'

CD = re.compile(r'\$ cd (\S+)')
LS = re.compile(r'\$ ls')
FILE = re.compile(r'(\d+) (\S+)')
DIR = re.compile(r'dir (\S+)')

def tree_to_str(node: Node, indent=1):
    s = f'{node}\n'
    for child in node.children:
        s += f'{"  " * indent}{tree_to_str(child, indent+1)}'
    return s

def build_tree(input: list[str]) -> Node:
    root_node = Directory('/', None)
    node = root_node
    i = 0
    while i < len(input):
        line = input[i]

        cd = CD.match(line)
        if cd is not None:
            target = cd.group(1)
            node = node.cd(target)
            i += 1
            continue
        
        ls = LS.match(line)
        if ls is not None:
            i += 1
            continue

        dir = DIR.match(line)
        if dir is not None:
            node.add_child(Directory(dir.group(1), node))
            i += 1
            continue

        file = FILE.match(line)
        if file is not None:
            node.add_child(File(file.group(2), node, int(file.group(1))))
            i += 1
            continue

    #print(tree_to_str(root_node))

    return root_node

def solution1(input: list[str]) -> int:
    root_node = build_tree(input)

    dirs = root_node.collect_subdirectories() + [root_node]

    return sum(map(lambda d: d.size, filter(lambda d: d.size <= 100000, dirs)))


In [105]:

def solution2(input: list[str]) -> int:
    root_node = build_tree(input)

    dirs = root_node.collect_subdirectories() + [root_node]
    dirs.sort(key=lambda n: n.size)

    total_space = 70_000_000
    available_space = total_space - root_node.size
    needed_space = 30_000_000 - available_space
    
    for dir in dirs:
        if dir.size > needed_space:
            return dir.size


In [106]:
solutions = [
    solution1,
    solution2,
]

test_results = [
    get_int_from_file('./test_result1'),
    get_int_from_file('./test_result2'),
]

def run_test(idx) -> bool:
    res = solutions[idx-1](get_test_input())
    test_res = test_results[idx-1]
    
    if test_res == res:
        print(f'Your solution for part {idx} works!!! :) (on the test input, that is)')
        print(f'Let`s try it on the actual input now...')
        return True
    else:
        print(f'Your solution for part {idx} does not work yet. Keep going!')
        print(f'You`ve got {res}, but the correct test result is {test_res}')
        return False

def run_solution(idx):
    sol = solutions[idx-1](get_input())
    print(f'The solution for part {idx} is: {sol}')

if run_test(1):
    run_solution(1)
    print('\nOn to part 2...\n')
    if run_test(2):
        run_solution(2)

Your solution for part 1 works!!! :) (on the test input, that is)
Let`s try it on the actual input now...
The solution for part 1 is: 1642503

On to part 2...

Your solution for part 2 works!!! :) (on the test input, that is)
Let`s try it on the actual input now...
The solution for part 2 is: 6999588
